# Convex Hull

![convex hull](https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/ConvexHull.svg/440px-ConvexHull.svg.png)

## Convex Hull Problem Description

The Convex Hull problem is a fundamental problem in computational geometry that deals with finding the smallest convex polygon that contains all the points in a given set. In simple terms, imagine you have a set of points on a plane, and you want to create a rubber band that stretches around these points such that it encloses all of them. When the rubber band is released, it will contract and wrap around the outermost points, forming the convex hull of the set.

Mathematically, a set of points S in a Euclidean space is said to be convex if for every pair of points P and Q in S, the line segment connecting P and Q lies entirely within S. The convex hull is the smallest convex set containing all the points in the given set.

In computational geometry, the Convex Hull problem has several applications, such as collision detection in video games, path planning in robotics, and shape analysis in computer vision.

There are various algorithms to solve the Convex Hull problem, some of which are:

* Gift Wrapping Algorithm (Jarvis March): This algorithm starts with the leftmost point and iteratively selects the next point by wrapping around the set in a counter-clockwise direction. Its time complexity is O(nh), where n is the number of input points, and h is the number of points in the convex hull.

* Graham's Scan: This algorithm sorts the points by their polar angles and processes them in a stack to build the convex hull. Its time complexity is O(n log n), as the most time-consuming step is sorting the points.

* QuickHull: This algorithm is a recursive divide-and-conquer approach, similar to the QuickSort algorithm. It selects extreme points and divides the input set into subsets based on their position relative to the line connecting the extreme points. The time complexity of QuickHull is O(n log n) in the average case, but it can degrade to O(n^2) in the worst case.

* Chan's Algorithm: This algorithm is a combination of Graham's Scan and Jarvis March. It computes the convex hull in O(n log h) time, making it an optimal output-sensitive algorithm.



Each of these algorithms has its strengths and weaknesses, and the choice of which one to use depends on factors such as the input size, expected output size, and available computational resources.


## Graham's Scan

Graham's Scan is an efficient algorithm for finding the convex hull of a set of points in a 2D plane. It has a time complexity of O(n log n), where n is the number of input points. The algorithm follows these steps:

* Find the lowest y-coordinate point (P0):
Identify the point with the lowest y-coordinate among the input points. If there are multiple points with the same y-coordinate, choose the one with the lowest x-coordinate. This point (P0) is guaranteed to be a part of the convex hull.

* Sort the points based on their polar angles with respect to P0:
Calculate the polar angles of the remaining points with respect to P0. Sort the points in increasing order of these polar angles. In case of a tie (when two points have the same polar angle), order them based on their distance from P0, with the closest point coming first.

* Initialize the convex hull stack:
Create a stack data structure to store the points in the convex hull. Push the first three sorted points (P0, P1, and P2) onto the stack.

* Process the remaining sorted points:
Iterate through the remaining sorted points, starting from the fourth point (P3). For each point, perform the following steps:

1. Check the orientation of the triplet formed by the top two points in the stack and the current point. If the orientation is counter-clockwise (a left turn), it means the current point is a valid convex hull point. Push the current point onto the stack and proceed to the next point in the sorted list.

2. If the orientation is clockwise (a right turn) or collinear, it means the second-to-top point in the stack is not part of the convex hull, as it is "inside" the turn. Pop the top point off the stack and recheck the orientation of the new triplet formed by the top two points in the stack and the current point. Repeat this process until a counter-clockwise turn is encountered or the stack has only two points left. Then, push the current point onto the stack and continue to the next point in the sorted list.

Termination:
Once all the points have been processed, the points remaining in the stack form the vertices of the convex hull in counter-clockwise order.

The most time-consuming step in Graham's Scan is sorting the points based on their polar angles, which takes O(n log n) time. The actual construction of the convex hull using the stack has a linear complexity of O(n). Therefore, the overall time complexity of the algorithm is O(n log n).

In [2]:
import math

def lowest_y_coordinate(points):
    return min(points, key=lambda point: (point[1], point[0]))

def polar_angle(p0, p1):
    return math.atan2(p1[1] - p0[1], p1[0] - p0[0])

def distance(p0, p1):
    return (p1[0] - p0[0])**2 + (p1[1] - p0[1])**2

def orientation(p0, p1, p2):
    return (p1[1] - p0[1]) * (p2[0] - p0[0]) - (p1[0] - p0[0]) * (p2[1] - p0[1])

def graham_scan(points):
    '''This implementation takes a list of 2D points as input, 
    where each point is represented as a tuple of (x, y) coordinates.
      It returns a list of points representing the vertices of the convex hull in counter-clockwise order.

Please note that this code does not handle duplicate points or points with the same polar angle. 
To handle these edge cases, you can modify the sorting step accordingly.'''
    if len(points) < 3:
        return points

    # Step 1: Find the lowest y-coordinate point
    lowest_point = lowest_y_coordinate(points)
    
    # Step 2: Sort the points based on their polar angles with respect to the lowest_point
    sorted_points = sorted(points, key=lambda point: (polar_angle(lowest_point, point), distance(lowest_point, point)))
    
    # Step 3: Initialize the convex hull stack
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]

    # Step 4: Process the remaining sorted points
    for i in range(3, len(sorted_points)):
        while len(convex_hull) >= 2 and orientation(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0:
            convex_hull.pop()
        convex_hull.append(sorted_points[i])

    return convex_hull

# Example usage:
points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2)]
convex_hull = graham_scan(points)
print(convex_hull)

[(1, 1), (2, 5)]


In [5]:
points = [(1, 1), (1, 5), (5, 1), (5, 5), (3, 3), (2, 2)]
convex_hull = graham_scan(points)
print(convex_hull)

[(1, 1), (5, 1), (2, 2), (1, 5)]


## Improved Graham's Scan to handle collinear points



In [6]:
import math

def lowest_y_coordinate(points):
    return min(points, key=lambda point: (point[1], point[0]))

def polar_angle(p0, p1):
    return math.atan2(p1[1] - p0[1], p1[0] - p0[0])

def distance(p0, p1):
    return (p1[0] - p0[0])**2 + (p1[1] - p0[1])**2

def orientation(p0, p1, p2):
    return (p1[1] - p0[1]) * (p2[0] - p0[0]) - (p1[0] - p0[0]) * (p2[1] - p0[1])

def graham_scan(points):
    if len(points) < 3:
        return points

    # Step 1: Find the lowest y-coordinate point
    lowest_point = lowest_y_coordinate(points)
    
    # Step 2: Sort the points based on their polar angles and distance with respect to the lowest_point
    sorted_points = sorted(points, key=lambda point: (polar_angle(lowest_point, point), distance(lowest_point, point)))

    # Remove duplicate points
    sorted_points = [sorted_points[i] for i in range(len(sorted_points)) if i == 0 or sorted_points[i] != sorted_points[i - 1]]
    
    # Step 3: Initialize the convex hull stack
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]

    # Step 4: Process the remaining sorted points
    for i in range(3, len(sorted_points)):
        while len(convex_hull) >= 2 and orientation(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0:
            convex_hull.pop()
        convex_hull.append(sorted_points[i])

    return convex_hull

# Example usage:
points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2), (3, 3), (5, 3)]
convex_hull = graham_scan(points)
print(convex_hull)

[(1, 1), (2, 5)]


## Visualiation of Graham's Scan


In [9]:
import plotly.graph_objs as go

# def lowest_y_coordinate(points):
#     # ... (same as before)

# def polar_angle(p0, p1):
#     # ... (same as before)

# def distance(p0, p1):
#     # ... (same as before)

# def orientation(p0, p1, p2):
#     # ... (same as before)

def graham_scan_trace(points):
    if len(points) < 3:
        return points, []

    # Step 1: Find the lowest y-coordinate point
    lowest_point = lowest_y_coordinate(points)
    
    # Step 2: Sort the points based on their polar angles and distance with respect to the lowest_point
    sorted_points = sorted(points, key=lambda point: (polar_angle(lowest_point, point), distance(lowest_point, point)))

    # Remove duplicate points
    sorted_points = [sorted_points[i] for i in range(len(sorted_points)) if i == 0 or sorted_points[i] != sorted_points[i - 1]]
    
    # Step 3: Initialize the convex hull stack
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]
    convex_hull_trace = []

    # Step 4: Process the remaining sorted points
    for i in range(3, len(sorted_points)):
        while len(convex_hull) >= 2 and orientation(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0:
            convex_hull.pop()
        convex_hull.append(sorted_points[i])
        convex_hull_trace.append(go.Scatter(x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
                                            y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
                                            mode="lines",
                                            line=dict(color="gray", dash="dash"),
                                            name=f"Step {i+1}"))

    return convex_hull, convex_hull_trace

def visualize(points, convex_hull, convex_hull_trace):
    input_points = go.Scatter(
        x=[point[0] for point in points],
        y=[point[1] for point in points],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Input Points",
    )

    hull_points = go.Scatter(
        x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
        y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
        mode="lines+markers",
        marker=dict(size=8, color="red"),
        line=dict(color="red"),
        name="Convex Hull",
    )

    traces = [input_points, hull_points] + convex_hull_trace
    layout = go.Layout(title="Graham's Scan Algorithm Visualization", showlegend=True)

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

points = [(1, 1), (2, 5), (2.5, 3), (5, 3), (3, 2), (2, 2), (3, 3.5), (5, 3)]
convex_hull, convex_hull_trace = graham_scan_trace(points)
visualize(points, convex_hull, convex_hull_trace)

## Jarvis March - aka - Gift Wrapping Algorithm


Jarvis March, also known as the Gift Wrapping algorithm, is a method for finding the convex hull of a set of points in a 2D plane. It is called the Gift Wrapping algorithm because its process is similar to wrapping a gift with a piece of string by iteratively stretching it around the outermost points of the gift. The algorithm has a time complexity of O(nh), where n is the number of input points and h is the number of points in the convex hull.

Here's a detailed description of how the Jarvis March algorithm works:

* Find the leftmost point: Start by identifying the point with the lowest x-coordinate, which is guaranteed to be part of the convex hull. If there are multiple points with the same x-coordinate, choose the one with the lowest y-coordinate. This point is referred to as the "anchor" point.

* Initialize the current point: Set the current point as the anchor point.

* Find the next hull point: For each point in the input set, excluding the current point, determine the point that forms the smallest clockwise angle (also known as the smallest polar angle) with the current point and the positive x-axis (or the line segment connecting the current point and the previous hull point, after the first iteration). In other words, find the point that would create a right turn if we were to move from the previous hull point to the current point and then to the candidate point. The point with the smallest clockwise angle becomes the next hull point.

* Check for termination: If the next hull point is the anchor point, the algorithm terminates, and the convex hull is complete. If not, set the current point as the next hull point and repeat step 3.

* Convex hull: The sequence of points visited in the clockwise direction forms the convex hull.

The Jarvis March algorithm is relatively simple to understand and implement. However, its time complexity depends on the number of points in the convex hull (h). In the worst case, when all points are part of the convex hull, the algorithm has a time complexity of O(n^2). This makes it less efficient than other algorithms like Graham's Scan or Chan's Algorithm for larger input sets or when the convex hull has many points. However, Jarvis March can be more efficient for cases where the convex hull has a small number of points, regardless of the total number of input points.



![Gift](https://upload.wikimedia.org/wikipedia/commons/9/9c/Animation_depicting_the_gift_wrapping_algorithm.gif)

In [10]:
def leftmost_point(points):
    return min(points, key=lambda point: (point[0], point[1]))

def orientation(p0, p1, p2):
    return (p1[1] - p0[1]) * (p2[0] - p0[0]) - (p1[0] - p0[0]) * (p2[1] - p0[1])

def jarvis_march(points):
    if len(points) < 3:
        return points

    # Step 1: Find the leftmost point
    anchor = leftmost_point(points)

    # Initialize the convex hull
    convex_hull = [anchor]

    # Step 2: Initialize the current point
    current_point = anchor

    while True:
        # Step 3: Find the next hull point
        next_point = None
        for candidate_point in points:
            if candidate_point == current_point:
                continue
            if next_point is None or orientation(current_point, next_point, candidate_point) > 0:
                next_point = candidate_point

        # Step 4: Check for termination
        if next_point == anchor:
            break

        # Update the current point and add the next hull point to the convex hull
        current_point = next_point
        convex_hull.append(next_point)

    return convex_hull

# Example usage:
points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2)]
convex_hull = jarvis_march(points)
print(convex_hull)

[(1, 1), (5, 3), (2, 5)]


In [11]:
points = [(1, 1), (1, 5), (3, 3), (5, 5), (2, 3), (5,1)]
convex_hull = jarvis_march(points)
print(convex_hull)

[(1, 1), (5, 1), (5, 5), (1, 5)]


## Visualizing the Jarvis March Algorithm


In [12]:
def jarvis_march_trace(points):
    if len(points) < 3:
        return points, []

    # ... (same as before)
    anchor = leftmost_point(points)
    convex_hull = [anchor]
    current_point = anchor
    convex_hull_trace = []

    while True:
        next_point = None
        for candidate_point in points:
            if candidate_point == current_point:
                continue
            if next_point is None or orientation(current_point, next_point, candidate_point) > 0:
                next_point = candidate_point

        if next_point == anchor:
            break

        current_point = next_point
        convex_hull.append(next_point)
        convex_hull_trace.append(go.Scatter(x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
                                            y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
                                            mode="lines",
                                            line=dict(color="gray", dash="dash"),
                                            name=f"Step {len(convex_hull)}"))

    return convex_hull, convex_hull_trace

def visualize(points, convex_hull, convex_hull_trace):
    input_points = go.Scatter(
        x=[point[0] for point in points],
        y=[point[1] for point in points],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Input Points",
    )

    hull_points = go.Scatter(
        x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
        y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
        mode="lines+markers",
        marker=dict(size=8, color="red"),
        line=dict(color="red"),
        name="Convex Hull",
    )

    traces = [input_points, hull_points] + convex_hull_trace
    layout = go.Layout(title="Jarvis March Algorithm Visualization", showlegend=True)

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2)]
convex_hull, convex_hull_trace = jarvis_march_trace(points)
visualize(points, convex_hull, convex_hull_trace)

In [14]:
points = [(1, 1), (2, 3), (4, 5), (5, 7), (6, 1), (2, 2),(3,3),(3,0.5)]
convex_hull, convex_hull_trace = jarvis_march_trace(points)
visualize(points, convex_hull, convex_hull_trace)